# Collaborative filtering

In [17]:
#librairies
# classic Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import shutil

#loading embeddings
import pickle

#sklearn utils
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import NearestNeighbors

#surprise
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split #really different from sklearn's ? 
from surprise import accuracy

In [18]:
#check if .ipynb files have been generated : len should be 385
path = "data/archive/clicks/clicks"
list_of_files = os.listdir(path)
print(len(list_of_files))

385


In [19]:
#concat all files in one
dfs = []
for file in list_of_files:
    file_path = os.path.join(path, file)
    dfs.append(pd.read_csv(file_path))
clicks_full = pd.concat(dfs, ignore_index=True)

In [20]:
#print("Full size data set :", clicks_full.shape)

In [21]:
#loading sample
data_sample = pd.read_csv("data/archive/clicks_sample.csv")
data_sample.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


In [22]:
'''#create rating
ratings = (
    data_sample
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)
'''

'#create rating\nratings = (\n    data_sample\n    .groupby(["user_id","click_article_id"])\n    .size()\n    .reset_index(name="rating")\n)\n'

In [23]:
#ratings.head()

In [24]:
'''reader = Reader(rating_scale=(1, ratings.rating.max()))
data   = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)'''

'reader = Reader(rating_scale=(1, ratings.rating.max()))\ndata   = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)\ntrainset, testset = train_test_split(data, test_size=0.2, random_state=42)'

In [25]:
#print(ratings["rating"].value_counts().sort_index())

In [26]:
'''algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)
preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))'''

'algo = SVD(n_factors=50, random_state=42)\nalgo.fit(trainset)\npreds = algo.test(testset)\nprint("RMSE:", accuracy.rmse(preds))\nprint("MAE: ", accuracy.mae(preds))'

In [27]:
#EDA
# 2.2 Aperçu
print(data_sample.shape)
display(data_sample.head())
print(data_sample.info())

# 2.3 Valeurs manquantes / doublons
print("Nulls :", data_sample.isna().sum().sum())
print("Doublons :", data_sample.duplicated().sum())

# 2.4 Nombre d’utilisateurs / articles
print("Users uniques :", data_sample["user_id"].nunique())
print("Articles uniques :", data_sample["click_article_id"].nunique())


(1883, 12)


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   user_id              1883 non-null   int64
 1   session_id           1883 non-null   int64
 2   session_start        1883 non-null   int64
 3   session_size         1883 non-null   int64
 4   click_article_id     1883 non-null   int64
 5   click_timestamp      1883 non-null   int64
 6   click_environment    1883 non-null   int64
 7   click_deviceGroup    1883 non-null   int64
 8   click_os             1883 non-null   int64
 9   click_country        1883 non-null   int64
 10  click_region         1883 non-null   int64
 11  click_referrer_type  1883 non-null   int64
dtypes: int64(12)
memory usage: 176.7 KB
None
Nulls : 0
Doublons : 0
Users uniques : 707
Articles uniques : 323


In [28]:
'''# 3.2 Agréger en comptant les clics par (user, article)
ratings = (
    data_sample
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)

print(ratings.shape)
display(ratings.head())
print(ratings["rating"].describe())
'''

'# 3.2 Agréger en comptant les clics par (user, article)\nratings = (\n    data_sample\n    .groupby(["user_id","click_article_id"])\n    .size()\n    .reset_index(name="rating")\n)\n\nprint(ratings.shape)\ndisplay(ratings.head())\nprint(ratings["rating"].describe())\n'

In [29]:
'''from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# 4.1 Création de l’objet Dataset
reader = Reader(rating_scale=(ratings["rating"].min(), ratings["rating"].max()))
data = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)

# 4.2 Split train/test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 4.3 Entraînement d’un SVD de base
algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

# 4.4 Évaluation
preds = algo.test(testset)
print("RMSE :", accuracy.rmse(preds))
print("MAE  :", accuracy.mae(preds))
'''

'from surprise import Dataset, Reader, SVD\nfrom surprise.model_selection import train_test_split\nfrom surprise import accuracy\n\n# 4.1 Création de l’objet Dataset\nreader = Reader(rating_scale=(ratings["rating"].min(), ratings["rating"].max()))\ndata = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)\n\n# 4.2 Split train/test\ntrainset, testset = train_test_split(data, test_size=0.2, random_state=42)\n\n# 4.3 Entraînement d’un SVD de base\nalgo = SVD(n_factors=50, random_state=42)\nalgo.fit(trainset)\n\n# 4.4 Évaluation\npreds = algo.test(testset)\nprint("RMSE :", accuracy.rmse(preds))\nprint("MAE  :", accuracy.mae(preds))\n'

In [30]:
'''from collections import defaultdict
import numpy as np

def precision_recall_at_k(predictions, k=5, threshold=1.0):
    # Regrouper les prédictions par user
    user_pred = defaultdict(list)
    user_true = defaultdict(set)
    for uid, iid, true_r, est, _ in predictions:
        user_pred[uid].append((iid, est))
        if true_r >= threshold:
            user_true[uid].add(iid)

    precisions, recalls = [], []
    for uid, preds in user_pred.items():
        # top-k par score estimé
        top_k = [iid for (iid, _) in sorted(preds, key=lambda x: x[1], reverse=True)[:k]]
        n_rel = len(user_true[uid])
        n_rec_k = len([i for i in top_k if i in user_true[uid]])
        precisions.append(n_rec_k / k)
        recalls.append(n_rec_k / n_rel if n_rel else 0)

    return np.mean(precisions), np.mean(recalls)

prec, rec = precision_recall_at_k(preds, k=5, threshold=1.0)
print(f"Precision@5 = {prec:.4f}, Recall@5 = {rec:.4f}")
'''

'from collections import defaultdict\nimport numpy as np\n\ndef precision_recall_at_k(predictions, k=5, threshold=1.0):\n    # Regrouper les prédictions par user\n    user_pred = defaultdict(list)\n    user_true = defaultdict(set)\n    for uid, iid, true_r, est, _ in predictions:\n        user_pred[uid].append((iid, est))\n        if true_r >= threshold:\n            user_true[uid].add(iid)\n\n    precisions, recalls = [], []\n    for uid, preds in user_pred.items():\n        # top-k par score estimé\n        top_k = [iid for (iid, _) in sorted(preds, key=lambda x: x[1], reverse=True)[:k]]\n        n_rel = len(user_true[uid])\n        n_rec_k = len([i for i in top_k if i in user_true[uid]])\n        precisions.append(n_rec_k / k)\n        recalls.append(n_rec_k / n_rel if n_rel else 0)\n\n    return np.mean(precisions), np.mean(recalls)\n\nprec, rec = precision_recall_at_k(preds, k=5, threshold=1.0)\nprint(f"Precision@5 = {prec:.4f}, Recall@5 = {rec:.4f}")\n'

In [31]:
data_meta = pd.read_csv("data/archive/articles_metadata.csv")
# 1. Vérification de la correspondance des IDs
click_ids = set(clicks_full["click_article_id"].unique())
meta_ids  = set(data_meta["article_id"].unique())

n_click_ids   = len(click_ids)
n_meta_ids    = len(meta_ids)
n_common_ids  = len(click_ids & meta_ids)

print(f"IDs uniques dans clicks : {n_click_ids}")
print(f"IDs uniques dans meta   : {n_meta_ids}")
print(f"IDs en commun           : {n_common_ids} ({n_common_ids/n_click_ids:.1%} des IDs de clics)")

# 2. IDs présents dans clicks mais pas dans meta
missing_in_meta = click_ids - meta_ids
print(f"IDs en clicks mais pas en meta : {len(missing_in_meta)}")
print("Quelques exemples :", list(missing_in_meta)[:10])

# 3. IDs présents dans meta mais pas dans clicks
missing_in_clicks = meta_ids - click_ids
print(f"IDs en meta mais pas en clicks : {len(missing_in_clicks)}")
print("Quelques exemples :", list(missing_in_clicks)[:10])

IDs uniques dans clicks : 46033
IDs uniques dans meta   : 364047
IDs en commun           : 46033 (100.0% des IDs de clics)
IDs en clicks mais pas en meta : 0
Quelques exemples : []
IDs en meta mais pas en clicks : 318014
Quelques exemples : [0, 1, 2, 4, 5, 6, 7, 8, 9, 10]


Which implies that users have read the same articles multiples times, and that some articles have never been read

Sounds strange and inaccurate

Retrain now with full dataset

In [33]:
'''# 3.2 Agréger en comptant les clics par (user, article)
ratings = (
    clicks_full
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)

print(ratings.shape)
display(ratings.head())
print(ratings["rating"].describe())'''

'# 3.2 Agréger en comptant les clics par (user, article)\nratings = (\n    clicks_full\n    .groupby(["user_id","click_article_id"])\n    .size()\n    .reset_index(name="rating")\n)\n\nprint(ratings.shape)\ndisplay(ratings.head())\nprint(ratings["rating"].describe())'

In [34]:
'''ratings["rating"].unique()'''

'ratings["rating"].unique()'

In [35]:
'''#4(Optionnel) Calculer un dwell_time pour un rating pondéré
clicks_full = clicks_full.sort_values(
    ["user_id","session_id","click_timestamp"]
)
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id","session_id"])["click_timestamp"]
    .shift(-1)
)
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)

# 5. Construire le DataFrame de ratings sur counts
ratings_count = (
    clicks_full
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)
print(ratings_count["rating"].describe())

# 6. Ou bien sur dwell_time
ratings_time = (
    clicks_full
    .groupby(["user_id","click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="rating")
)
print(ratings_time["rating"].describe())'''

'#4(Optionnel) Calculer un dwell_time pour un rating pondéré\nclicks_full = clicks_full.sort_values(\n    ["user_id","session_id","click_timestamp"]\n)\nclicks_full["next_ts"] = (\n    clicks_full\n    .groupby(["user_id","session_id"])["click_timestamp"]\n    .shift(-1)\n)\nclicks_full["dwell_time"] = (\n    clicks_full["next_ts"] - clicks_full["click_timestamp"]\n).fillna(0)\n\n# 5. Construire le DataFrame de ratings sur counts\nratings_count = (\n    clicks_full\n    .groupby(["user_id","click_article_id"])\n    .size()\n    .reset_index(name="rating")\n)\nprint(ratings_count["rating"].describe())\n\n# 6. Ou bien sur dwell_time\nratings_time = (\n    clicks_full\n    .groupby(["user_id","click_article_id"])["dwell_time"]\n    .sum()\n    .reset_index(name="rating")\n)\nprint(ratings_time["rating"].describe())'

In [36]:
'''from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Choix du rating
ratings = ratings_count  # ou ratings_time

reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data   = Dataset.load_from_df(
    ratings[["user_id","click_article_id","rating"]],
    reader
)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))
'''

'from surprise import Dataset, Reader, SVD\nfrom surprise.model_selection import train_test_split\nfrom surprise import accuracy\n\n# Choix du rating\nratings = ratings_count  # ou ratings_time\n\nreader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))\ndata   = Dataset.load_from_df(\n    ratings[["user_id","click_article_id","rating"]],\n    reader\n)\ntrainset, testset = train_test_split(data, test_size=0.2, random_state=42)\n\nalgo = SVD(n_factors=50, random_state=42)\nalgo.fit(trainset)\n\npreds = algo.test(testset)\nprint("RMSE:", accuracy.rmse(preds))\nprint("MAE: ", accuracy.mae(preds))\n'

In [37]:
'''#dwell time

# 2. Calculer le dwell_time (temps passé sur chaque article au sein de chaque session)
clicks_full = clicks_full.sort_values(
    ["user_id", "session_id", "click_timestamp"]
)

# Pour chaque clic, on prend le timestamp du clic suivant dans la même session
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id", "session_id"])["click_timestamp"]
    .shift(-1)
)

# dwell_time = différence au clic suivant, ou 0 si c'est le dernier clic de la session
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)

# 3. Construire le DataFrame de ratings sur la somme des dwell_time par (user, article)
ratings_time = (
    clicks_full
    .groupby(["user_id", "click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="rating")
)

# (Optionnel) Appliquer une transformation log+1 pour atténuer les valeurs extrêmes
ratings_time["rating"] = np.log1p(ratings_time["rating"])

# 4. Charger ces ratings dans Surprise et entraîner un modèle SVD
reader = Reader(rating_scale=(ratings_time["rating"].min(), ratings_time["rating"].max()))
data   = Dataset.load_from_df(
    ratings_time[["user_id", "click_article_id", "rating"]],
    reader
)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

# 5. Évaluer sur le testset (RMSE / MAE)
preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))

# 6. Exemple de prédiction pour un utilisateur & un article donné
#    Format : algo.predict(uid, iid, r_ui=None, verbose=True)
#    - uid = ID utilisateur (doit exister dans trainset ou non ; Surprise peut faire du cold-start)
#    - iid = ID article (doit exister dans trainset ou non)
#    - r_ui = rating réel si connu, sinon None
#    - verbose = True pour afficher la prédiction détaillée

user_id    = 1234    # remplacer par un user_id de ton choix
article_id = 157541  # remplacer par un click_article_id de ton choix

prediction = algo.predict(uid=user_id, iid=article_id, r_ui=None, verbose=True)
# prediction.est est la valeur prédite (log(dwell_time+1)) pour ce couple (user_id, article_id)

# Si tu veux récupérer la valeur sous forme de dwell_time estimé, fais :
log_est = prediction.est
dwell_time_est = np.expm1(log_est)  # inverse de np.log1p
print(f"Estimation de dwell_time pour user {user_id}, article {article_id}: {dwell_time_est:.2f} ms")'''

'#dwell time\n\n# 2. Calculer le dwell_time (temps passé sur chaque article au sein de chaque session)\nclicks_full = clicks_full.sort_values(\n    ["user_id", "session_id", "click_timestamp"]\n)\n\n# Pour chaque clic, on prend le timestamp du clic suivant dans la même session\nclicks_full["next_ts"] = (\n    clicks_full\n    .groupby(["user_id", "session_id"])["click_timestamp"]\n    .shift(-1)\n)\n\n# dwell_time = différence au clic suivant, ou 0 si c\'est le dernier clic de la session\nclicks_full["dwell_time"] = (\n    clicks_full["next_ts"] - clicks_full["click_timestamp"]\n).fillna(0)\n\n# 3. Construire le DataFrame de ratings sur la somme des dwell_time par (user, article)\nratings_time = (\n    clicks_full\n    .groupby(["user_id", "click_article_id"])["dwell_time"]\n    .sum()\n    .reset_index(name="rating")\n)\n\n# (Optionnel) Appliquer une transformation log+1 pour atténuer les valeurs extrêmes\nratings_time["rating"] = np.log1p(ratings_time["rating"])\n\n# 4. Charger ces r

In [38]:
'''import pickle

# Define save path
save_path = os.path.join("models_in_progress", "cf_model.pkl")

# Serialize
with open(save_path, "wb") as f_out:
    pickle.dump(algo, f_out)

print(os.listdir("models_in_progress"))
'''

'import pickle\n\n# Define save path\nsave_path = os.path.join("models_in_progress", "cf_model.pkl")\n\n# Serialize\nwith open(save_path, "wb") as f_out:\n    pickle.dump(algo, f_out)\n\nprint(os.listdir("models_in_progress"))\n'

nb de click par user sur un article / pondère par activité user lors de sa session (nb total de click par session)

par user's session  : \
nb total de click \
nb de click par article

temps passé sur chaque article \
nb de click sur un même lien

## second CB, more implicit

In [41]:
# 1) On calcule, pour chaque clic, le timestamp du clic suivant DANS LA MÊME SESSION
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id", "session_id"])["click_timestamp"]
    .shift(-1)
)
# 2) On calcule le dwell_time : temps écoulé entre clic courant et clic suivant
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)


C:\Users\kanam\AppData\Local\Temp\ipykernel_21556\3556110278.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


In [42]:
# 3) On agrège le dwell_time par couple (user_id, click_article_id)
dwell_per_pair = (
    clicks_full
    .groupby(["user_id", "click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="total_dwell_time")
)


In [43]:
'''clicks_full = clicks_full.sort_values(
    ["user_id", "session_id", "click_timestamp"]
)
'''

'clicks_full = clicks_full.sort_values(\n    ["user_id", "session_id", "click_timestamp"]\n)\n'

In [44]:
'''dwell_per_pair["dwell_score"] = np.log1p(dwell_per_pair["total_dwell_time"])
'''

'dwell_per_pair["dwell_score"] = np.log1p(dwell_per_pair["total_dwell_time"])\n'

In [45]:
# session_size est déjà donnée dans le dataset (nombre de clics dans la session)
# Si ce n’était pas le cas, on ferait :
# session_counts = (
#   clicks_full.groupby(["user_id","session_id"])
#   .size().reset_index(name="session_size")
# )


In [46]:
# 4) On calcule, pour chaque ligne, la proportion de clics sur le même article DANS LA SESSION
clicks_full["clicks_same_session"] = (
    clicks_full
    .groupby(["user_id","session_id","click_article_id"])
    ["click_article_id"]
    .transform("count")
)


In [47]:

clicks_full["session_weight"] = (
    clicks_full["clicks_same_session"] / clicks_full["session_size"]
)


In [48]:
# 5) On agrège ce poids RELATIF par couple (user, article)
session_weight_per_pair = (
    clicks_full
    .groupby(["user_id", "click_article_id"])["session_weight"]
    .sum()
    .reset_index(name="total_session_weight")
)

In [49]:
# 6) On compte dans combien de SESSIONS distinctes l'utilisateur a cliqué sur l'article
sessions_per_pair = (
    clicks_full
    .drop_duplicates(subset=["user_id", "session_id", "click_article_id"])
    .groupby(["user_id", "click_article_id"])
    .size()
    .reset_index(name="n_sessions_clicked")
)


In [50]:
print(type(session_weight_per_pair["total_session_weight"]))            # doit être <class 'pandas.core.series.Series'>
print(session_weight_per_pair["total_session_weight"].head())          # un aperçu des 5 premières valeurs
print(session_weight_per_pair["total_session_weight"].dtype)           # dtype, ex. float64


<class 'pandas.core.series.Series'>
0    0.5
1    0.5
2    0.5
3    0.5
4    0.5
Name: total_session_weight, dtype: object
object


In [51]:
# 7) (Optionnel) On transforme en log pour atténuer l'effet des grosses valeurs
vals = session_weight_per_pair["total_session_weight"].to_numpy(dtype=float)
session_weight_per_pair["sw_score"] = np.log1p(vals)

dwell_per_pair["dwell_score"] = np.log1p(dwell_per_pair["total_dwell_time"])
#session_weight_per_pair["sw_score"] = np.log1p(session_weight_per_pair["total_session_weight"])
sessions_per_pair["sc_score"] = np.log1p(sessions_per_pair["n_sessions_clicked"])

#sessions_per_pair["session_count_score"] = np.log1p(sessions_per_pair["n_sessions_clicked"])


In [52]:
'''# 4.1. rename des colonnes pour homogénéité
dwell_per_pair = dwell_per_pair.rename(
    columns={"click_article_id": "article_id"}
)

session_weight_per_pair = session_weight_per_pair.rename(
    columns={"click_article_id": "article_id", "total_session_weight": "sw_score"}
)

sessions_per_pair = sessions_per_pair.rename(
    columns={"click_article_id": "article_id", "n_sessions_clicked": "n_sess", "session_count_score": "sc_score"}
)

# 4.2. Merge successifs
ratings_imp = dwell_per_pair.merge(
    session_weight_per_pair,
    on=["user_id","article_id"],
    how="outer"
).merge(
    sessions_per_pair,
    on=["user_id","article_id"],
    how="outer"
)

# 4.3. Remplir les NaN (si un utilisateur n’avait jamais de dwell_time ou n’apparaît que dans l’une des deux tables)
ratings_imp = ratings_imp.fillna(0)'''


'# 4.1. rename des colonnes pour homogénéité\ndwell_per_pair = dwell_per_pair.rename(\n    columns={"click_article_id": "article_id"}\n)\n\nsession_weight_per_pair = session_weight_per_pair.rename(\n    columns={"click_article_id": "article_id", "total_session_weight": "sw_score"}\n)\n\nsessions_per_pair = sessions_per_pair.rename(\n    columns={"click_article_id": "article_id", "n_sessions_clicked": "n_sess", "session_count_score": "sc_score"}\n)\n\n# 4.2. Merge successifs\nratings_imp = dwell_per_pair.merge(\n    session_weight_per_pair,\n    on=["user_id","article_id"],\n    how="outer"\n).merge(\n    sessions_per_pair,\n    on=["user_id","article_id"],\n    how="outer"\n)\n\n# 4.3. Remplir les NaN (si un utilisateur n’avait jamais de dwell_time ou n’apparaît que dans l’une des deux tables)\nratings_imp = ratings_imp.fillna(0)'

In [53]:
# 8) On fusionne ces trois DataFrames sur (user_id, article_id)
from functools import reduce

dfs = [
    dwell_per_pair.rename(columns={"click_article_id":"article_id"}),
    session_weight_per_pair.rename(columns={"click_article_id":"article_id"}),
    sessions_per_pair.rename(columns={"click_article_id":"article_id"})
]
ratings_imp = reduce(
    lambda left, right: left.merge(right, on=["user_id","article_id"], how="left"),
    dfs
).fillna(0)


C:\Users\kanam\AppData\Local\Temp\ipykernel_21556\16259381.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


In [54]:
ratings_imp

,user_id,article_id,total_dwell_time,dwell_score,total_session_weight,sw_score,n_sessions_clicked,sc_score
0,0,68866,0,0.000000,0.5,0.405465,1,0.693147
1,0,87205,0,0.000000,0.5,0.405465,1,0.693147
2,0,87224,30000,10.308986,0.5,0.405465,1,0.693147
3,0,96755,30000,10.308986,0.5,0.405465,1,0.693147
4,0,157541,30000,10.308986,0.5,0.405465,1,0.693147
...,...,...,...,...,...,...,...,...
2950705,322894,168401,0,0.000000,0.5,0.405465,1,0.693147
2950706,322895,63746,0,0.000000,0.5,0.405465,1,0.693147
2950707,322895,289197,30000,10.308986,0.5,0.405465,1,0.693147
2950708,322896,30760,30000,10.308986,0.5,0.405465,1,0.693147


In [55]:
print(ratings_imp.columns.tolist())


['user_id', 'article_id', 'total_dwell_time', 'dwell_score', 'total_session_weight', 'sw_score', 'n_sessions_clicked', 'sc_score']


In [56]:
# 9) Normalisation Min-Max de chacune de ces trois composantes
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

ratings_imp["dwell_norm"] = scaler.fit_transform(ratings_imp[["dwell_score"]])
ratings_imp["sw_norm"]    = scaler.fit_transform(ratings_imp[["sw_score"]])
ratings_imp["sc_norm"]    = scaler.fit_transform(ratings_imp[["sc_score"]])


In [57]:
'''from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# On normalise dwell_score (qui est déjà log+1)
ratings_imp["dwell_norm"] = scaler.fit_transform(
    ratings_imp[["dwell_score"]]
)

# On normalise sw_score
ratings_imp["sw_norm"] = scaler.fit_transform(
    ratings_imp[["sw_score"]]
)

# On normalise sc_score
ratings_imp["sc_norm"] = scaler.fit_transform(
    ratings_imp[["sc_score"]]
)

# Puis on combine
w1, w2, w3 = 0.5, 0.3, 0.2
ratings_imp["rating_imp"] = (
    w1 * ratings_imp["dwell_norm"]
  + w2 * ratings_imp["sw_norm"]
  + w3 * ratings_imp["sc_norm"]
)
'''

'from sklearn.preprocessing import MinMaxScaler\n\nscaler = MinMaxScaler()\n\n# On normalise dwell_score (qui est déjà log+1)\nratings_imp["dwell_norm"] = scaler.fit_transform(\n    ratings_imp[["dwell_score"]]\n)\n\n# On normalise sw_score\nratings_imp["sw_norm"] = scaler.fit_transform(\n    ratings_imp[["sw_score"]]\n)\n\n# On normalise sc_score\nratings_imp["sc_norm"] = scaler.fit_transform(\n    ratings_imp[["sc_score"]]\n)\n\n# Puis on combine\nw1, w2, w3 = 0.5, 0.3, 0.2\nratings_imp["rating_imp"] = (\n    w1 * ratings_imp["dwell_norm"]\n  + w2 * ratings_imp["sw_norm"]\n  + w3 * ratings_imp["sc_norm"]\n)\n'

In [58]:
# 10) Combinaison pondérée en un seul rating implicite
w1, w2, w3 = 0.5, 0.3, 0.2
ratings_imp["rating_imp"] = (
    w1 * ratings_imp["dwell_norm"]
  + w2 * ratings_imp["sw_norm"]
  + w3 * ratings_imp["sc_norm"]
)


In [59]:
# 11) (Optionnel) Mise à l’échelle pour Surprise, p.ex. dans [1,5]
ratings_imp["rating_final"] = 1 + 4 * ratings_imp["rating_imp"]


In [60]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(
    ratings_imp[["user_id","article_id","rating_final"]],
    reader
)


In [61]:
# 1. Charger le dataset complet de clics
clicks_dir = "data/archive/clicks/clicks"
files = [f for f in os.listdir(clicks_dir) if f.endswith(".csv")]
dfs = [pd.read_csv(os.path.join(clicks_dir, f)) for f in files]
clicks_full = pd.concat(dfs, ignore_index=True)

In [62]:
# 2. Calcul du dwell_time
clicks_full = clicks_full.sort_values(
    ["user_id","session_id","click_timestamp"]
)
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id","session_id"])["click_timestamp"]
    .shift(-1)
)
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)

# 3. Calcul du dwell_score = log1p(total_dwell_time)
dwell_per_pair = (
    clicks_full
    .groupby(["user_id","click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="total_dwell_time")
)
dwell_per_pair["dwell_score"] = np.log1p(dwell_per_pair["total_dwell_time"])

# 4. Calcul du poids par session (session_weight)
clicks_full["clicks_same_session"] = (
    clicks_full
    .groupby(["user_id","session_id","click_article_id"])
    ["click_article_id"]
    .transform("count")
)
clicks_full["session_weight"] = (
    clicks_full["clicks_same_session"] / clicks_full["session_size"]
)

session_weight_per_pair = (
    clicks_full
    .groupby(["user_id","click_article_id"])["session_weight"]
    .sum()
    .reset_index(name="sw_score")
)

# 5. Calcul de la fréquence multi-session (sc_score)
sessions_per_pair = (
    clicks_full.drop_duplicates(
        subset=["user_id","session_id","click_article_id"]
    )
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="n_sessions_clicked")
)
sessions_per_pair["sc_score"] = np.log1p(sessions_per_pair["n_sessions_clicked"])

# 6. Fusion des métriques
dwell2 = dwell_per_pair.rename(columns={"click_article_id":"article_id"})
sw2    = session_weight_per_pair.rename(columns={"click_article_id":"article_id"})
sc2    = sessions_per_pair.rename(columns={"click_article_id":"article_id", "n_sessions_clicked":"n_sess"})

ratings_imp = dwell2.merge(sw2, on=["user_id","article_id"], how="outer") \
                    .merge(sc2, on=["user_id","article_id"], how="outer")

ratings_imp = ratings_imp.fillna(0)

# 7. Normalisation Min-Max de chaque score
scaler = MinMaxScaler()
ratings_imp["dwell_norm"] = scaler.fit_transform(
    ratings_imp[["dwell_score"]]
)
ratings_imp["sw_norm"]    = scaler.fit_transform(
    ratings_imp[["sw_score"]]
)
ratings_imp["sc_norm"]    = scaler.fit_transform(
    ratings_imp[["sc_score"]]
)

# 8. Combinaison pondérée
w1, w2, w3 = 0.5, 0.3, 0.2
ratings_imp["rating_imp"] = (
    w1 * ratings_imp["dwell_norm"]
  + w2 * ratings_imp["sw_norm"]
  + w3 * ratings_imp["sc_norm"]
)

# 9. Mise à l’échelle finale (1 à 5) pour Surprise
ratings_imp["rating_final"] = 1 + 4 * ratings_imp["rating_imp"]

ratings_imp.head()


C:\Users\kanam\AppData\Local\Temp\ipykernel_21556\3181132234.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)
C:\Users\kanam\AppData\Local\Temp\ipykernel_21556\3181132234.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings_imp = ratings_imp.fillna(0)


,user_id,article_id,total_dwell_time,dwell_score,sw_score,n_sess,sc_score,dwell_norm,sw_norm,sc_norm,rating_imp,rating_final
0,0,68866,0,0.000000,0.5,1,0.693147,0.000000,0.089574,0.0,0.026872,1.107489
1,0,87205,0,0.000000,0.5,1,0.693147,0.000000,0.089574,0.0,0.026872,1.107489
2,0,87224,30000,10.308986,0.5,1,0.693147,0.492884,0.089574,0.0,0.273314,2.093256
3,0,96755,30000,10.308986,0.5,1,0.693147,0.492884,0.089574,0.0,0.273314,2.093256
4,0,157541,30000,10.308986,0.5,1,0.693147,0.492884,0.089574,0.0,0.273314,2.093256


In [63]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(
    ratings_imp[["user_id","article_id","rating_final"]],
    reader
)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

algo_cf = SVD(n_factors=50, random_state=42)
algo_cf.fit(trainset)

preds = algo_cf.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))


RMSE: 0.5164
RMSE: 0.5163591623626093
MAE:  0.4752
MAE:  0.4751698691392017


In [64]:
demo_user = ratings_imp["user_id"].iloc[0]
print(f"Exemple d'utilisateur : {demo_user}")
demo_article = ratings_imp["article_id"].iloc[0]
print(f"Exemple d'article : {demo_article}")

algo_cf.predict(demo_user, demo_article)

Exemple d'utilisateur : 0
Exemple d'utilisateur : 68866


Prediction(uid=0, iid=68866, r_ui=None, est=1.5178435034919495, details={'was_impossible': False})

In [65]:
data_meta.columns

Index(['article_id', 'category_id', 'created_at_ts', 'publisher_id',
       'words_count'],
      dtype='object')

In [66]:
# 1. Choix du user_id pour la démo (par exemple, le premier de ratings_imp)
demo_user = ratings_imp["user_id"].iloc[0]
print(f"Utilisateur de démonstration : {demo_user}")

# 2. Historique CF : lister les articles déjà vus par ce user
user_seen = set(ratings_imp.loc[
    ratings_imp["user_id"] == demo_user, "article_id"
].tolist())
print(f"Articles déjà cliqués ({len(user_seen)}): {list(user_seen)[:5]}…")

# 3. Préparer le pool de candidats CF : tous les article_id de data_meta non vus
all_articles = set(data_meta["article_id"].tolist())
candidates = list(all_articles - user_seen)
print(f"Nombre de candidats CF possibles : {len(candidates)}")

# 4. Prédire un score CF pour chaque candidat
cf_algo = algo_cf

cf_preds = []
for iid in candidates:
    pred = cf_algo.predict(uid=demo_user, iid=iid, r_ui=None, verbose=False)
    cf_preds.append((iid, pred.est))

# 5. Trier par score décroissant et prendre le top-k
k = 5
topk_cf = sorted(cf_preds, key=lambda x: x[1], reverse=True)[:k]
topk_ids = [iid for iid, _ in topk_cf]
topk_scores = [score for _, score in topk_cf]

# 6. Construire un DataFrame résultat et joindre les métadonnées
df_cf = pd.DataFrame({
    "article_id": topk_ids,
    "score_cf":   topk_scores
})
df_cf = df_cf.merge(
    data_meta,
    on="article_id",
    how="left"
)[[
    "article_id","category_id","publisher_id","words_count","created_at_ts","score_cf"
]]

# 7. Afficher les recommandations CF pures
print("Top 5 recommandations CF pures :")
display(df_cf)


Utilisateur de démonstration : 0
Articles déjà cliqués (8): [68866, 157541, 87205, 313996, 96755]…
Nombre de candidats CF possibles : 364039
Top 5 recommandations CF pures :


,article_id,category_id,publisher_id,words_count,created_at_ts,score_cf
0,257732,391,0,127,1507648925000,2.063084
1,194199,317,0,185,1507975860000,2.056109
2,227849,374,0,166,1507500412000,2.022434
3,314286,431,0,161,1507466177000,2.006254
4,136553,255,0,113,1507625576000,1.989056


In [68]:
# Define save path
save_path = os.path.join("models_in_progress", "cf_model.pkl")

# Serialize
with open(save_path, "wb") as f_out:
    pickle.dump(cf_algo, f_out)

print(os.listdir("models_in_progress"))

['cf_model.pkl']


In [72]:
"""# 1) Création et fit sur training set
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(ratings_imp[["rating_final"]])

# Define save path
save_path = os.path.join("models_in_progress", "cf_scaler.pkl")

# 2) Sauvegarde
with open(save_path, "wb") as f:
    pickle.dump(scaler, f)"""
#this is stupid because data € [1,5] so scaler can be retrained at every cold start without much resource consumption